In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import StratifiedKFold

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
import math

In [4]:
df = pd.read_csv('C:/Users/Lenovo/Documents/Documents/Data_Set/Iris.csv',index_col=['Id'])
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
ord_rank = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}
df['Species_ordinal'] = df['Species'].map(ord_rank)
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Species_ordinal
Id,,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa,0
2,4.9,3.0,1.4,0.2,Iris-setosa,0
3,4.7,3.2,1.3,0.2,Iris-setosa,0
4,4.6,3.1,1.5,0.2,Iris-setosa,0
5,5.0,3.6,1.4,0.2,Iris-setosa,0


In [6]:
X = df.iloc[:,:-2]
y = df.iloc[:,-1]

In [7]:
print(X.head())
print(y.head())

    SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
Id                                                          
1             5.1           3.5            1.4           0.2
2             4.9           3.0            1.4           0.2
3             4.7           3.2            1.3           0.2
4             4.6           3.1            1.5           0.2
5             5.0           3.6            1.4           0.2
Id
1    0
2    0
3    0
4    0
5    0
Name: Species_ordinal, dtype: int64


In [8]:
std = StandardScaler()
X = std.fit_transform(X)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X,y,random_state=23, test_size=0.3)
def toDataFrame(X):
    return pd.DataFrame(X)
x_train = toDataFrame(x_train)
x_test = toDataFrame(x_test)
y_train = toDataFrame(y_train)
y_test = toDataFrame(y_test)

In [10]:
x_train.shape

(105, 4)

In [11]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [12]:
def Stacking(model,x_train, x_test, y_test,y_train, nfold):
    folds = StratifiedKFold(n_splits=nfold,shuffle=True, random_state=23)
    test_pred = list()
    
    train_pred = list()
    
    y_train_pred = list()
    #here we do not need Y-test-pred because we using same x_test to predict y
   # y_test_pred = list()
    
    for train_indices, val_indices in folds.split(x_train, y_train.values):
        
        x_train_ = x_train.iloc[list(train_indices),:] 
        x_val = x_train.iloc[list(val_indices),:]
        y_train_ = y_train.iloc[train_indices]
        y_val = y_train.iloc[val_indices]
        
        model.fit(x_train_, y_train_.values.ravel())
        #print('Y Train',y_train_.values.ravel().shape)
        #print('X Train',x_train_.shape)
        #print(list(model.predict(x_val)))
        train_pred.extend(list(model.predict(x_val).ravel()))
        #dd the predicted model which is list of shape x_test.shape to 
        #o test_pred 
        test_pred.append(list(model.predict(x_test).ravel()))      
        y_train_pred.extend(list(y_val.values.ravel()))
        
        #now test_pred is of size  (nfold, x_test.shape[0])
        #since columns in test_pred contain the predicted values by 
        #by different k-values from k-flod so we need to take mean of each column 
        # so as to get mean value in each column of the nfold results
        #so here mean are from each column
    
    return train_pred, np.array(pd.DataFrame(np.array(test_pred)).apply(lambda x: math.floor(np.mean(x)))), y_train_pred
    

In [13]:
X.shape

(150, 4)

## Model 1 Decision Tree

In [14]:
model_decision = DecisionTreeClassifier()
train_pred1, test_pred1, y_train_pred1 = Stacking(model=model_decision, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test,nfold=10)

In [15]:
print(len(train_pred1))

105


In [16]:
x_train.shape

(105, 4)

In [17]:
test_pred1

array([2, 2, 1, 0, 2, 1, 0, 2, 0, 1, 1, 0, 2, 0, 0, 1, 1, 1, 2, 0, 2, 0,
       0, 0, 2, 0, 0, 1, 1, 1, 0, 1, 0, 2, 0, 0, 1, 1, 1, 2, 2, 0, 1, 0,
       1], dtype=int64)

In [18]:
toDataFrame(np.array(a)).apply(lambda x: math.floor(np.mean(x)))
# here I have combine 10 or nfold test result output mean from 10 results of datasize x_test.shape

NameError: name 'a' is not defined

In [ ]:
print(x_test.shape)
print(test_pred1.shape)
print(x_train.shape)
print('X_train_pred',len(train_pred1))
print('Y_train_pred',len(y_train_pred1))


## Model 2 KNN

In [19]:
model_knn = KNeighborsClassifier(n_neighbors=35)
train_pred2, test_pred2, y_train_pred2 = Stacking(model_knn, x_train, x_test, y_test, y_train, 10)

In [20]:
print(x_test.shape)
print(test_pred2.shape)
print(x_train.shape)
print('X_train_pred',len(train_pred2))
print('Y_train_pred',len(y_train_pred2))
# here y_train_pred is validated nfolds indices of kfold method

(45, 4)
(45,)
(105, 4)
X_train_pred 105
Y_train_pred 105


In [21]:
train_pred1 = pd.DataFrame(train_pred1)
train_pred2 = pd.DataFrame(train_pred2)

In [22]:
test_pred1 = pd.DataFrame(test_pred1)
test_pred2 = pd.DataFrame(test_pred2)

In [23]:
train_pred1.head()


,0
0,1
1,0
2,2
3,2
4,1


In [24]:
train_pred2.head()

,0
0,1
1,0
2,2
3,2
4,1


In [25]:
x_train_predicted_stacked = pd.concat([train_pred1,train_pred2], axis=1)
x_train_predicted_stacked.head()

,0,0
0,1,1
1,0,0
2,2,2
3,2,2
4,1,1


In [26]:
x_pedicted_test_stacked = pd.concat([test_pred1, test_pred2], axis=1)
x_pedicted_test_stacked.head()

,0,0
0,2,2
1,2,2
2,1,1
3,0,0
4,2,1


In [27]:
y_train_pred1==y_train_pred2

True

In [28]:
logit_model = LogisticRegression(max_iter=1000)
logit_model.fit(x_train_predicted_stacked, y_train_pred1)

LogisticRegression(max_iter=1000)

## AS we can see our result has been improved using Stacking from 91% to 95.5%

In [29]:
logit_model.score(x_pedicted_test_stacked, y_test)

0.9555555555555556

In [30]:
print(y_train_pred2)
print(y_train_pred1)

[1, 0, 2, 2, 1, 2, 2, 1, 1, 0, 0, 0, 2, 0, 1, 2, 1, 2, 2, 1, 0, 1, 0, 2, 0, 2, 1, 2, 1, 1, 1, 0, 2, 2, 1, 1, 0, 0, 2, 1, 1, 2, 2, 0, 2, 1, 1, 2, 1, 2, 0, 0, 2, 0, 1, 2, 2, 2, 0, 2, 0, 0, 1, 1, 1, 1, 1, 2, 0, 1, 0, 0, 2, 2, 2, 2, 2, 1, 1, 2, 1, 0, 0, 0, 0, 2, 0, 1, 1, 0, 2, 1, 0, 2, 0, 2, 0, 0, 1, 2, 1, 0, 1, 2, 1]
[1, 0, 2, 2, 1, 2, 2, 1, 1, 0, 0, 0, 2, 0, 1, 2, 1, 2, 2, 1, 0, 1, 0, 2, 0, 2, 1, 2, 1, 1, 1, 0, 2, 2, 1, 1, 0, 0, 2, 1, 1, 2, 2, 0, 2, 1, 1, 2, 1, 2, 0, 0, 2, 0, 1, 2, 2, 2, 0, 2, 0, 0, 1, 1, 1, 1, 1, 2, 0, 1, 0, 0, 2, 2, 2, 2, 2, 1, 1, 2, 1, 0, 0, 0, 0, 2, 0, 1, 1, 0, 2, 1, 0, 2, 0, 2, 0, 0, 1, 2, 1, 0, 1, 2, 1]


### np.empty create empty array but with junks

In [31]:
n= np.empty((55,1),dtype=int)

In [32]:
n=np.append(n,[1,2,3,57,8])
n

array([ -29930312,        293,        186,          0,          0,
                0,          0,          0,          0,    6029423,
       1868767867,  975332708, 1869768226, 1802707053, 1918985580,
       1768697454, 1918985582, 1685024095, 1763732581, 1919905901,
       1867260020, 1953720679, 1699898217, 1936028263, 1852795251,
       1919315548, 1931505007, 1634036843, 1831759474, 1818584175,
       1818587999, 1769235301, 1763733103, 1919905901, 1920213108,
       1601071457, 1953719668, 1819308895, 1851552873, 1836020326,
       1818981152, 1852989797, 1701998638, 1668248176, 1769173861,
       1763731310, 1919905901, 1951604852, 1633971809, 1666409586,
       1919249505, 1696738338, 1969448312, 1852795252, 1970234207,
                1,          2,          3,         57,          8])

In [33]:
np.append(n, [1,324,546,54,65,76,8,7,8,798,654,65,45,35,3,43,423,4,234,325,4,6])

array([ -29930312,        293,        186,          0,          0,
                0,          0,          0,          0,    6029423,
       1868767867,  975332708, 1869768226, 1802707053, 1918985580,
       1768697454, 1918985582, 1685024095, 1763732581, 1919905901,
       1867260020, 1953720679, 1699898217, 1936028263, 1852795251,
       1919315548, 1931505007, 1634036843, 1831759474, 1818584175,
       1818587999, 1769235301, 1763733103, 1919905901, 1920213108,
       1601071457, 1953719668, 1819308895, 1851552873, 1836020326,
       1818981152, 1852989797, 1701998638, 1668248176, 1769173861,
       1763731310, 1919905901, 1951604852, 1633971809, 1666409586,
       1919249505, 1696738338, 1969448312, 1852795252, 1970234207,
                1,          2,          3,         57,          8,
                1,        324,        546,         54,         65,
               76,          8,          7,          8,        798,
              654,         65,         45,         35,        